In [1]:
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from textblob import TextBlob 
from tqdm import tqdm

In [2]:
client = pymongo.Connection()
collection = client.politics.debates

In [3]:
texts = collection.find()
df = pd.DataFrame(list(texts))

In [4]:
df

,_id,days_after,debate,newspaper,text,title,word_count,year
0,1,0,1,Washington Post,EIGHT Questions\nWhat's at stake in the first ...,EIGHT Questions,2215,2012
1,10,2,1,The International Herald Tribune,﻿\nDebate shakes up presidential race;\nRomney...,Debate shakes up presidential race,1234,2012
2,100,1,2,The Gazette,﻿\nMore aggressive Obama goes on offensive; Pr...,More aggressive Obama goes on offensive,888,2012
3,101,1,3,The Telegraph,﻿\nBarack Obama labels Romney reckless in heat...,Barack Obama labels Romney reckless in heated ...,2539,2012
4,102,1,3,The Telegraph,White House showdown sees Obama label Mitt Rom...,White House showdown sees Obama label Mitt Rom...,2539,2012
5,103,1,3,Washington Post,"BOCA RATON, Fla. - President Obama and Mitt Ro...",No Title,1659,2012
6,104,1,3,The New York Post,﻿\nAmerica Decides Desperate Bam 'bombs' Romne...,America Decides Desperate Bam 'bombs' Romney B...,1115,2012
7,105,0,3,Hollywood Reporter,﻿\nObama-Romney Final Debate: What the Pundits...,Obama-Romney Final Debate: What the Pundits Ar...,1096,2012
8,106,0,3,Washington Post,"﻿\nSuddenly, close race on foreign policy\nIt ...","Suddenly, close race on foreign policy",1747,2012
9,107,1,3,Pittsburgh Post-Gazette,﻿\nROUND : TWO WEEKS LEFT;\nCANDIDATES SHIFT F...,Round 3: Two Weeks Left,1518,2012


In [5]:
def calcSentiment(text):
    blob = TextBlob(text)
    x = [sentence.sentiment.polarity for sentence in blob.sentences]
    
    return(x)

In [16]:
df["text_sentiment"] = df.text.apply(calcSentiment)

In [9]:
def nNeg(sentiments):
    s = [x for x in sentiments if x < 0]
    return len(s)

In [22]:
df["per_neg"] = df.text_sentiment.apply(nNeg)/len(df["text_sentiment"])*100

In [23]:
df.groupby(["debate", "days_after"]).mean()

_id   word_count  year    per_neg
debate days_after                                          
1      0            17.600000  1639.000000  2012   7.866667
       1            27.357143  1166.785714  2012   7.619048
       2            24.764706  1204.411765  2012   7.921569
2      0            78.000000  1044.333333  2012   6.888889
       1            76.076923  1012.269231  2012   6.641026
       2            74.428571  1258.380952  2012   7.873016
3      0           129.750000  1143.916667  2012   7.666667
       1           123.862069  1243.586207  2012  10.482759
       2           125.111111  1144.000000  2012   8.888889

In [17]:
df.head()

,_id,days_after,debate,newspaper,text,title,word_count,year,text_sentiment
0,1,0,1,Washington Post,EIGHT Questions\nWhat's at stake in the first ...,EIGHT Questions,2215,2012,"[0.25, 0.0, 0.0, 0.25, 0.16666666666666666, -0..."
1,10,2,1,The International Herald Tribune,﻿\nDebate shakes up presidential race;\nRomney...,Debate shakes up presidential race,1234,2012,"[0.21875, 0.09999999999999999, 0.5, 0.0, 0.0, ..."
2,100,1,2,The Gazette,﻿\nMore aggressive Obama goes on offensive; Pr...,More aggressive Obama goes on offensive,888,2012,"[0.5, 0.16666666666666666, 0.1708333333333333,..."
3,101,1,3,The Telegraph,﻿\nBarack Obama labels Romney reckless in heat...,Barack Obama labels Romney reckless in heated ...,2539,2012,"[-0.125, -0.5, 0.18571428571428575, 0.0, 0.166..."
4,102,1,3,The Telegraph,White House showdown sees Obama label Mitt Rom...,White House showdown sees Obama label Mitt Rom...,2539,2012,"[-0.15625, -0.5, 0.18571428571428575, 0.0, 0.1..."


In [30]:
x = TextBlob("More aggressive Obama goes on offensive")

In [33]:
x.sentiment.polarity

0.5

In [32]:
blob = TextBlob(df.text)
x = [sentence.sentiment.polarity for sentence in blob.sentences]


TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'pandas.core.series.Series'>

In [35]:
x.ngrams(n=3)

[WordList(['More', 'aggressive', 'Obama']),
 WordList(['aggressive', 'Obama', 'goes']),
 WordList(['Obama', 'goes', 'on']),
 WordList(['goes', 'on', 'offensive'])]

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = df.text[:4].tolist() + ["The words in this frame describe situations in which an Agent or a Cause injures a Victim. The Body_part of the Victim which is most directly affected may also be mentioned in the place of the Victim. In such cases, the Victim is often indicated as a genitive modifier of the Body_part, in which case the Victim FE is indicated on a second FE layer. "]
tfidf = TfidfVectorizer().fit_transform(documents)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T


In [63]:
(pairwise_similarity).toarray()


array([[ 1.        ,  0.76798554,  0.78067706,  0.77019167,  0.36533145],
       [ 0.76798554,  1.        ,  0.72013541,  0.77207068,  0.35165273],
       [ 0.78067706,  0.72013541,  1.        ,  0.77517085,  0.32464428],
       [ 0.77019167,  0.77207068,  0.77517085,  1.        ,  0.31501399],
       [ 0.36533145,  0.35165273,  0.32464428,  0.31501399,  1.        ]])

In [66]:
print(df.text[3])

﻿
Barack Obama labels Romney reckless in heated TV clash as White House race
enters final lap
US President Barack Obama snatched the advantage of the incumbency in the
foreign policy debate with Republican Mitt Romney in Boca Raton, Florida last
night, accusing him of advocating positions that were "wrong and reckless".
Looking him in the eye, the president said: "Every time you've offered an
opinion you've been wrong".
For Mr Romney, who seemed less at ease than in the previous two face-offs and
more than once found himself  agreeing with stances taken by the president, it
was a night mostly of avoiding howlers in an area of American leadership on
which he is necessarily less experienced.  Burnish his candidacy, however, he
probably did not.
If Mr Obama was the fiercer of the two it was because his task was more urgent -
to stem the recent shift in the polls towards his foe including in some key
battleground states.  Mocking a lament from Mr Romney that the US Navy is down
to  ships f